In [159]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
import time
import random
import csv
import pandas as pd

In [ ]:
# 필수 입력
area_select = ['서울', '경기'] # 전국 / 서울, 경기, 인천, 부산, 대구, 광주, 대전, 울산, 세종, 강원, 경남, 경북, 전남, 전북, 충북, 제주
company_name = '유한'


In [138]:
driver = webdriver.Chrome(executable_path=r'/Users/hyeseon/Documents/chromedriver')
driver.get("https://www.saramin.co.kr/zf_user/")


In [139]:
#검색창 clear
elem = driver.find_element_by_id('ipt_keyword_recruit')
elem.clear()

#검색어 입력
elem.send_keys(company_name)


In [140]:
# 검색클릭
elem = driver.find_element_by_id("btn_search_recruit")
elem.click()

In [141]:
cha = ['Y', 'Y', 'N']

In [142]:
# 채용정보 더보기 후 첫페이지로 이동
elem = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[2]/div/a')
elem.click()


In [143]:
elem = driver.find_element_by_xpath('//*[@id="recruit_info_list"]/div[2]/div/a[1]')
elem.click()

In [144]:
# 처음 100개를 받아오기
elem = driver.find_element_by_xpath("//*[@id='recruit_info']/div[2]/div/div[3]/button")
elem.click()


In [145]:
elem = driver.find_element_by_xpath("//*[contains(text(), '100개씩')]")
elem.click()


In [147]:
# 지역 선택으로 받을 것
elem = driver.find_element_by_class_name('area_section')
elem.click()

# 지역 펼쳐보기
elem = driver.find_element_by_class_name('btn_all_category')
elem.click()


In [148]:
area = driver.find_element_by_xpath('//*[@id="sp_main_wrapper"]/div[2]/div/div[2]/div[2]/div[1]/div[2]/div/ul[1]')
area = area.find_elements_by_tag_name("li")


In [156]:
area_list = []
xpath_list = []
for j in range(len(area)):
    area_id = area[j].get_attribute("id")
    
    if area_id == "":
        continue

    xpath = '//*[@id="'+area_id+'"]'
    title = area[j].find_element_by_tag_name('span').text
    area_list.append(title)
    xpath_list.append([xpath, title])


In [163]:
area_path_table = pd.DataFrame(xpath_list, columns =['xpath', 'area'])
area_path_table['selected']  = area_path_table['area'].map(lambda x : 1 if '전국' in area_select else 1 if x in area_select else 0)

In [67]:
# 선택된 지역 기준으로 채용 공고를 선택하게 만들기.

In [8]:
# 일반 채용정보 / 파견.대행 / 헤드헌팅 중 선택하는 경우를 리스트로 받음
# 전체 채용 정보일 경우 pass로
if cha == ['Y', 'Y', 'Y'] :
    pass

# 최소한 하나 이상의 채용 조건을 선택해야 함
if cha[0] + cha[1] + cha[2] == 'NNN' : 
    print('error')

# 일반 채용정보 선택 해제 
if cha[0] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[1]')
    elem.click()
    
if cha[1] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[2]')
    elem.click()
    
if cha[2] != 'Y' :
    elem = driver.find_element_by_xpath('//*[@id="recruit_info"]/div[1]/div/div[3]')
    elem.click()


In [162]:
driver.quit()